# Practice Text classification with Naive Bayes  
        
        
        
<h3>Abstract</h3>
<p>We will do text classification on a collection of Dutch parliamentary questions.
    The website <a href="https://zoek.officielebekendmakingen.nl/zoeken/parlementaire_documenten">officielebekendmakingen.nl</a>lets you search in "kamervragen".
    <!--You can donwload
    <a href='http://data.politicalmashup.nl/kamervragen/PoliDocs_Kamervragen.zip'>this zipfile with Kamervragen in XML</a>
    to see some of the  data in XML format. 
    It also contains style sheets to show the XML well in a browser.  
-->
    The <a href='http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/'>MYSQL directory</a> contains an <a href='http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR14807.xml'>example   Kamervraag XML file</a> and a file `kvr.csv.gz` with 40K kamervragen in a handy csv format. Note that in your browser you see the result of applying stylesheets. So choose View Source or open it in an editor.</p>

<h3>First exploration</h3>

See below.

<h2>Exercises</h2>

<p>We will use the fields in elements of the form <tt> &lt;item attribuut="Afkomstig_van"></tt> as our classes. 
    These are the ministeries to whom the question is addressed.
    An example is 
    <pre>
        &lt;item attribuut="Afkomstig_van">Landbouw, Natuurbeheer en Visserij (LNV)&lt;/item>
    </pre>
    Note that these labels are <strong>not normalized</strong>, see e.g. the counts below:
    <pre>
Justitie (JUS)                                                   3219
Volksgezondheid, Welzijn en Sport (VWS)                          2630
Buitenlandse Zaken (BUZA)                                        1796
Verkeer en Waterstaat (VW)                                       1441
Justitie                                                         1333
Sociale Zaken en Werkgelegenheid (SZW)                           1231
Onderwijs, Cultuur en Wetenschappen (OCW)                        1187
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer (VROM)     984
FinanciÃ«n (FIN)                                                   960
Volksgezondheid, Welzijn en Sport                                 951
Economische Zaken (EZ)                                            946
Buitenlandse Zaken                                                753
Binnenlandse Zaken en Koninkrijksrelaties (BZK)                   725
Verkeer en Waterstaat                                             724
Defensie (DEF)                                                    646
Sociale Zaken en Werkgelegenheid                                  607
Landbouw, Natuurbeheer en Visserij (LNV)                          586
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer            554
Onderwijs, Cultuur en Wetenschappen                               532
Vreemdelingenzaken en Integratie (VI)                             466
    </pre>
</p>


<h2>Form of handing in your final product</h2>

* An IPython notebook with for each question, a MarkDown cell containing the question, a code cell which solves the question, an output cell with the output, followed by a MarkDown cell with explanation/reflection  

In [1]:
import pandas as pd

names=['jaar', 'partij','titel','vraag','antwoord','ministerie']

# Change to KVR1000.csv.gz if this becomes too slow for you
# kvrdf= pd.read_csv('http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR.csv.gz', 
kvrdf= pd.read_csv('http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR1000.csv.gz', 
                   compression='gzip', sep='\t', 
                   index_col=0, names=names,
                   ) 

for kolom in names[1:]:
    kvrdf[kolom]= kvrdf[kolom].astype(str)
print(kvrdf.shape)
kvrdf.head()



(1000, 6)


,jaar,partij,titel,vraag,antwoord,ministerie
KVR1000.xml,1994,PvdA,De vragen betreffen de betrouwbaarheid van de...,Hebt u kennisgenomen van het televisieprogram...,Ja. Het bedoelde geluidmeetpunt is eigendom v...,Verkeer en Waterstaat
KVR10000.xml,1999,PvdA,Vragen naar aanleiding van berichten (uitzend...,Kent u de berichten over de situatie in de Me...,,Justitie
KVR10001.xml,1999,SP,"Vragen naar aanleiding van de berichten ""Nede...",Kent u de berichten «Nederland steunt de Soeh...,,Financien
KVR10002.xml,1999,PvdA,Vragen over de gebrekkige opvang van verpleeg...,Kent u het bericht over onderzoek van Nu91 me...,Ja. Het onderzoek van NU’91 wijst uit dat het...,"Volksgezondheid, Welzijn en Sport"
KVR10003.xml,1999,PvdA,Vragen over onbetrouwbaarheid van filemeldingen.,Hebt u kennisgenomen van de berichten over de...,Ja. Nee. Door de waarnemers van het Algemeen ...,Verkeer en Waterstaat


In [2]:
from nltk.corpus import stopwords
import nltk
from collections import Counter
import itertools
from math import log

DutchStop= stopwords.words('dutch')
allvragen= '\n'.join(list(kvrdf.titel))
classes = list(set(list(kvrdf.ministerie)))

Words are treated as lowercase & stopwords are filtered out below

In [3]:
# Term frequencies per word

def strip_string(string):
    """
    
    :param string: string of unfiltered word/symbols
    :return: list of all tokens extracted from the string (lowercasing, stopwords, alpha)
    """
    return [w for w in nltk.word_tokenize(string.lower()) if w.isalpha() and not w in set(DutchStop)]
    
def str_to_tf(string):
    """
    
    :param string: string of unfiltered word/symbols
    :return: dictionary of all term frequencies: occurance of term in string
    """
    return Counter(strip_string(string))





In [4]:

def str_to_tk(string):
    """ returns token count in a string
    """
    return len(strip_string(string))
    



## 1. 
Normalize the values for "ministerie" and choose 10 ministeries to work with.

In [5]:
# Only return classes that are about a single ministerie.
def determine_classes(classes):
    """ Remove any class that spans multiple ministeries
    
    :param classes: All classes that occur in kamervragen
    :return:
        norm_classes, a subset of classes with only the classes that span a single ministerie.
    """

    norm_classes = set()
    
    for c in classes:
        add = True

        if c == 'nan':
            continue

        for nc in norm_classes:
            if nc in c:
                add = False
                break
            elif c in nc:
                norm_classes.remove(nc)
                break

        if add:
            norm_classes.add(c)
            
    return norm_classes


# Normalize class c by replacing strange e's with a normal e
# and removing anything between parenthesis.
def normalize_class(c):
    """ Normalize class c by replacing different representations of the e by a normal e
    
    :param c: str, a single class name
    :return:
        str, a single class name with 'normal' e's and no trailing whitespace.
    """

    nc = ""
    parenthesis = False
    
    for char in c:
        if char == '(':
            parenthesis = True
            
        elif char == 'ë':
            char = 'e'
        elif char == 'Ã':
            char = 'e'
        elif char == '«':
            char = ''
            
        if not parenthesis:
            nc += char
            
        if char == ')':
            parenthesis = False
    
    return nc.strip()


def choose_10_classes(class_rows, norm_classes):
    """ Only return the 10 most occuring classes.
    
    :param class_rows: dict, a dictionary mapping a class name to all rows that class occurs in.
    :param norm_classes: set, all normalized class names of classes that only span a single ministerie.
    
    :return:
        set, the normalized names of the 10 most occuring classes
    """
    count = Counter()
    for key, rows in class_rows.items():
        key = normalize_class(key)
        
        if key in norm_classes:
            count[key] += len(rows)
            
    return set([name for name, _ in count.most_common(10)])


def kvrdf_to_10_classes(kvrdf, norm_classes):
    """ Return a copy of kvrdf with normalized class names and only containing rows with classes in norm_classes
    
    :param kvrdf: pd.Dataframe, containing all downloaded information: questions, ministerie, answers, etc.
    :param norm_classes: set, containing all normalized class names of the 10 most occuring classes.
    
    :return:
        pd.Dataframe, kvrdf but with all rows removed that don't contain any of the ministeries in norm_classes.
    """   
    all_classes = set(kvrdf.ministerie)
    
    for i, c in enumerate(all_classes):
        nc = normalize_class(c)
        
        if not (nc in norm_classes):
            kvrdf = kvrdf[kvrdf.ministerie != c]
            kvrdf.is_copy = False
            
        else:
            kvrdf.loc[kvrdf["ministerie"] == c, "ministerie"] = nc
            
    return kvrdf

# classes is a set of strings of all classes that are in kvrdf
classes = set(kvrdf.ministerie)

# norm classes is a set of strings of classes that only span a single ministerie with normalized names.
norm_classes = {normalize_class(c) for c in determine_classes(classes)}

# class_rows_full is a dictionary mapping a class name to all rows that contain that class.
class_rows_full = {c:kvrdf.loc[kvrdf.ministerie == c] for c in classes}

# classes is adjusted so it now is a set that contains the normalized strings of the 10 most occuring classes.
classes = choose_10_classes(class_rows_full, norm_classes)
print(classes)

# kvrdf is adjusted so any rows that don't contain any class in 'classes' are removed.
# also, all class names are normalized.
kvrdf = kvrdf_to_10_classes(kvrdf, classes)
kvrdf.head()

{'Buitenlandse Zaken', 'Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer', 'Verkeer en Waterstaat', 'Onderwijs, Cultuur en Wetenschappen', 'Justitie', 'Landbouw, Natuurbeheer en Visserij', 'Defensie', 'Economische Zaken', 'Sociale Zaken en Werkgelegenheid', 'Volksgezondheid, Welzijn en Sport'}


d:\uva\zoekmachines\zmsenv\lib\site-packages\pandas\core\generic.py:4388: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
d:\uva\zoekmachines\zmsenv\lib\site-packages\pandas\core\generic.py:4389: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)


,jaar,partij,titel,vraag,antwoord,ministerie
KVR1000.xml,1994,PvdA,De vragen betreffen de betrouwbaarheid van de...,Hebt u kennisgenomen van het televisieprogram...,Ja. Het bedoelde geluidmeetpunt is eigendom v...,Verkeer en Waterstaat
KVR10000.xml,1999,PvdA,Vragen naar aanleiding van berichten (uitzend...,Kent u de berichten over de situatie in de Me...,,Justitie
KVR10002.xml,1999,PvdA,Vragen over de gebrekkige opvang van verpleeg...,Kent u het bericht over onderzoek van Nu91 me...,Ja. Het onderzoek van NU’91 wijst uit dat het...,"Volksgezondheid, Welzijn en Sport"
KVR10003.xml,1999,PvdA,Vragen over onbetrouwbaarheid van filemeldingen.,Hebt u kennisgenomen van de berichten over de...,Ja. Nee. Door de waarnemers van het Algemeen ...,Verkeer en Waterstaat
KVR10004.xml,1999,D66,Vragen naar aanleiding van het bericht in de ...,Kent u het bericht als zou het RIVM onjuiste ...,Ja. Nee. Als reactie op het Volkskrant-artike...,Verkeer en Waterstaat


## 2.
Implement the two algorithms in Fig MRS.13.2, using your earlier code for creating term and document frequencies. It might be easier to use the representation and formula given in MRS section 13.4.1.

In [6]:

#document frequencies per class per term
def TrainMultinomialNB(pd_df, classes):
    """ more reference from p258 onwards of MRS
    
    :param classes: classes which are taken into account for training
    :param pd_df: kamervragen dataframe
    :return:
        V: set, of terms which which are used for classification
        class_priors: dictionary, prior probabilities for each class: P(c) prior[class]
        cond_prob: dictionary, conditional probabilities for each term per class: P(t|c) cond[class][term]
        dfdict: dictionary with document frequencies
        classes: list, classes actually used, temporary until class input is synchronised with pandas dataframe
    """
    class_frequency = Counter([c for c in pd_df.ministerie])
    
    
    # P(c)
    class_priors = {c:class_frequency[c]/sum([class_frequency[cn] for cn in classes]) for c in classes}

    # document frequencies per term
    k = [list(set(strip_string(t))) for t in kvrdf.titel]
    dfdict = Counter(list(itertools.chain.from_iterable(k)))

    #vocabulary
    V = set(dfdict.keys())
    
    # complete rows per class
    class_rows_full = {c:kvrdf.loc[kvrdf.ministerie == c] for c in classes}

    # term frequency per class
    class_tf = {c:str_to_tf('\n'.join(list(class_rows_full.get(c).titel))) for c in classes}
    
    # token count per class (for sum over Tct' in (t' in V), (p259, formula 13.6)
    class_tk = {c:str_to_tk('\n'.join(list(class_rows_full.get(c).titel))) for c in classes}

    #P(t|c)
    cond_prob = {t:{c:(class_tf[c][t] + 1)/(class_tk[c] + 1) for c in classes} for t in V}
    return V, class_priors, cond_prob, dfdict, classes

def filter_vocab(W, V):
    """ 
    
    :param W: list; stripped query
    :param V: set; of vocab used for classification (determined by one or anoter method)
    :return: a list of words that occur in the used vocabulary
    """
    return [w for w in W if w in V]

def ApplyMultinomialNB(classes, priors, conditionals, d, Vocab=None):
    """ Applies a Naive Bayes classifier on previously derived priors & conditionals on a query d

    :param classes: list, optional ministeries to classify between 
    :param Vocab: set, of terms which which are used for classification
    :param priors: dictionary, prior probabilities for each class: P(c) prior[class]
    :param conditionals: dictionary, conditional probabilities for each term per class: P(t|c) cond[class][term]
    :param d: string, query
    :return: best classification for query
    """
    W = strip_string(d)
    if Vocab:
        W = filter_vocab(W, Vocab)
    score = {c:log(priors[c]) for c in classes}
    for k in score.keys():
        score[k] += sum([log(conditionals[t][k]) for t in W])    
    best_class = sorted(score, key = lambda key: score[key])[0]
    print(sorted(score, key = lambda key: score[key]))
    return best_class



## 3.
On this collection, train NB text classifiers for 10 different classes with enough and interesting data.

In [7]:
V, class_priors, cond_prob, dfdict, classes = TrainMultinomialNB(kvrdf, classes)
ApplyMultinomialNB(classes, class_priors, cond_prob, "vragen")


['Economische Zaken', 'Defensie', 'Landbouw, Natuurbeheer en Visserij', 'Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer', 'Onderwijs, Cultuur en Wetenschappen', 'Sociale Zaken en Werkgelegenheid', 'Buitenlandse Zaken', 'Verkeer en Waterstaat', 'Volksgezondheid, Welzijn en Sport', 'Justitie']


'Economische Zaken'

## 4.
Compute for each term and each of your 10 classes its utility for that class using mutual information.

In [8]:
def extract_class_df(pddf, classes):
    class_dfdict = {c:{} for c in classes}

    for i, c in enumerate(classes):
        for d in kvrdf[kvrdf.ministerie == c].titel:
            for elem in set(strip_string(d)):
                if class_dfdict[c].get(elem, 0):
                    class_dfdict[c][elem] += 1
                else:
                    class_dfdict[c][elem] = 1
    return class_dfdict

cddf = extract_class_df(kvrdf, classes)

In [9]:
def MI(doc_freq, class_doc_freq):
    """ creates a mutual information dictionary for each term, class pair
    
    :param doc_freq: dictionary, contains documents frequencies per term, doc_freq[term]
    :param class_doc_freq: dictionary, contains documents frequencies per term per class, cdf[class][term]
    :return MIdict: dictionary, contains mutual information for each class/term pair, MI[class][term]
    """
    N = sum(doc_freq.values())
    MIdict = {}
    for c in class_doc_freq.keys():
        MIdict[c] = {}
        for t in class_doc_freq[c].keys():
            N11 = class_doc_freq[c][t]
            N10 = doc_freq[t] - N11
            N01 = sum(class_doc_freq[c].values()) - class_doc_freq[c][t]
            N00 = N - doc_freq[t] - (sum(class_doc_freq[c].values()) - class_doc_freq[c][t])
            N1 = N11 + N10
            N0 = N01 + N00
            if N11:
                MIdict[c][t] = N11/N * log( (N*N11)/(N1*N1), 2)
            if N01:
                MIdict[c][t] += N01/N * log( (N*N01)/(N0*N1) , 2)
            if N10:
                MIdict[c][t] += N10/N * log( (N*N10)/(N1*N0) , 2)
            if N00:
                MIdict[c][t] += N00/N * log(  (N*N00)/(N0*N0), 2)
    return MIdict

MIdict = MI(dfdict, cddf)

## 5.
For each class, show the top 10 words as in Figure 13.7 in MRS.

In [10]:
def top_MI(MIdict, N):
    """ 
    
    :param MIdict: a dictionary with mutual information for each MIdict[class][term]
    :param N: top N MI terms returned per class
    :return: a dictionary of the top N MI predictors of each class    
    """
    resdict = {}
    for c in MIdict.keys():
        resdict[c] = sorted(MIdict[c].items(), key = lambda key: -MIdict[c][key[0]])[:N]
    return resdict

def top_MI_to_markdown(topMI):
    firstline = classes_table_markdown(topMI)
    rest_of_table = values_table_markdown(topMI)
    return firstline + rest_of_table


def classes_table_markdown(topmidict):
    return "| []() | " + " | []() | ".join(topmidict.keys()) + "|\n" + "|" + " | ".join(['----- | -----' for _ in topmidict.keys()]) + "| \n"

def values_table_markdown(topmidict):
    returnstring = ""

    for i, elem in enumerate(topmidict[list(topmidict.keys())[0]]):
        returnstring += " | " + " | ".join([pair_to_markdown(topmidict[c][i]) for c in topmidict.keys()]) + " | \n"
    return returnstring
            
def pair_to_markdown(pair):
    return pair[0] + " | " + str(pair[1])


def topMI_to_vocab(topmidict):
    vocab_l_o_l = [[elem[0] for elem in topmidict[c]] for c in topmidict.keys()]
    return set(list(itertools.chain.from_iterable(vocab_l_o_l)))



In [11]:
topfeatures = top_MI(MIdict, 1)

dfdict2 = {t:v for t,v in dfdict.items() if v > 5}
cddf2 = {c:{t:cddf[c][t] for t in cddf[c].keys() if t in dfdict2.keys()} for c in cddf}
MIdict2 = MI(dfdict2, cddf2)
topfeatures2 = top_MI(MIdict, 85)


In [13]:
from IPython.display import display, Markdown, Latex


In [14]:
topfeatures_MD = top_MI_to_markdown(topfeatures)
topfeatures_MD2 = top_MI_to_markdown(topfeatures2)

# bug is because of empty classes & not being able to recognize classes in normalized form

display(Markdown(topfeatures_MD2))

| []() | Buitenlandse Zaken | []() | Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer | []() | Verkeer en Waterstaat | []() | Onderwijs, Cultuur en Wetenschappen | []() | Justitie | []() | Landbouw, Natuurbeheer en Visserij | []() | Defensie | []() | Economische Zaken | []() | Sociale Zaken en Werkgelegenheid | []() | Volksgezondheid, Welzijn en Sport|
|----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | ----- | -----| 
 | integriteit | 1.0017786861976063 | nutssectoren | 0.7415464892478193 | geluidsmetingen | 0.9064642091914905 | onderwijsinspecteurs | 0.7542116195739694 | blijt | 1.930452001280508 | subsidiëring | 0.3896423179577661 | fosforgranaten | 0.30255954304494675 | exploratie | 0.4346903879604562 | verkeert | 0.9046081268620453 | verpleegkundigen | 1.5588902321464466 | 
 | walsum | 1.0017786861976063 | handen | 0.7415464892478193 | filemeldingen | 0.9064642091914905 | onvrijwillige | 0.7542116195739694 | kliniek | 1.930452001280508 | haalbaarheidsstudie | 0.3896423179577661 | bodem | 0.30255954304494675 | clyde | 0.4346903879604562 | nadelige | 0.9046081268620453 | zorginstellingen | 1.5588902321464466 | 
 | balans | 1.0017786861976063 | voetnoten | 0.7415464892478193 | onbetrouwbaarheid | 0.9064642091914905 | enige | 0.7542116195739694 | twijfels | 1.930452001280508 | envirodung | 0.3896423179577661 | ton | 0.30255954304494675 | proefboringen | 0.4346903879604562 | achtergelaten | 0.9046081268620453 | traumatische | 1.5588902321464466 | 
 | territoriale | 1.0017786861976063 | toevoeging | 0.7415464892478193 | gepresenteerd | 0.9064642091914905 | inschatting | 0.7542116195739694 | professioneel | 1.930452001280508 | project | 0.3896423179577661 | oosterschelde | 0.30255954304494675 | petroleum | 0.4346903879604562 | arbeidsgehandicapte | 0.9046081268620453 | eilander | 1.5588902321464466 | 
 | anderzijds | 1.0017786861976063 | liberaal | 0.7415464892478193 | milieuaspecten | 0.9064642091914905 | vacature | 0.7542116195739694 | vervolgd | 1.930452001280508 | exportproblemen | 0.3896423179577661 | vlak | 0.30255954304494675 | plat | 0.4346903879604562 | werkplek | 0.9046081268620453 | vechten | 1.5588902321464466 | 
 | ambassadeur | 1.0017786861976063 | projectontwikkelaar | 0.7415464892478193 | onjuiste | 0.9064642091914905 | organisation | 0.7542116195739694 | patient | 1.930452001280508 | weyl | 0.3896423179577661 | zierikzee | 0.30255954304494675 | continentaal | 0.4346903879604562 | lisv | 0.9046081268620453 | waarneming | 1.5588902321464466 | 
 | respect | 1.0017786861976063 | utrechtse | 0.7415464892478193 | gehanteerd | 0.9064642091914905 | baccalaureate | 0.7542116195739694 | psychiatrische | 1.930452001280508 | firma | 0.3896423179577661 | haveningang | 0.30255954304494675 | gunnen | 0.4346903879604562 | loonkostensubsidie | 0.9046081268620453 | veldhuis | 1.5588902321464466 | 
 | souvereiniteit | 1.0017786861976063 | voorkeursrecht | 0.7415464892478193 | tunnels | 0.9064642091914905 | ondersteunende | 0.7542116195739694 | betaalgedrag | 1.930452001280508 | dioxinevrij | 0.3896423179577661 | przzn | 0.30255954304494675 | koreaans | 0.4346903879604562 | langere | 0.9046081268620453 | aflopen | 1.5588902321464466 | 
 | vrijheden | 1.0017786861976063 | grondeigenaar | 0.7415464892478193 | vervuild | 0.9064642091914905 | ibo | 0.7542116195739694 | persoonlijke | 1.930452001280508 | beef | 0.3896423179577661 | benutten | 0.30255954304494675 | platform | 0.4346903879604562 | beschikbaarheid | 0.9046081268620453 | declareert | 1.5588902321464466 | 
 | veiligheidsraad | 1.0017786861976063 | milieuhygiene | 0.7415464892478193 | bestemming | 0.9064642091914905 | krantenberichten | 0.7542116195739694 | omkoping | 1.930452001280508 | expliciete | 0.3896423179577661 | soldaten | 0.30255954304494675 | produktieplatform | 0.4346903879604562 | gehandicaptenvervoer | 0.9046081268620453 | verrichting | 1.5588902321464466 | 
 | debat | 1.0017786861976063 | agrarisch | 0.7415464892478193 | boortechniek | 0.9064642091914905 | onderwijstaak | 0.7542116195739694 | geldstromen | 1.930452001280508 | products | 0.3896423179577661 | vijfhonderd | 0.30255954304494675 | consortium | 0.4346903879604562 | hogere | 0.9046081268620453 | specialist | 1.5588902321464466 | 
 | oppakken | 1.0017786861976063 | gemanipuleerde | 0.7415464892478193 | boren | 0.9064642091914905 | stagebemiddeling | 0.7542116195739694 | grip | 1.930452001280508 | dioxinecrisis | 0.3896423179577661 | proef | 0.30255954304494675 | hanze | 0.4346903879604562 | reistijden | 0.9046081268620453 | overeenkomsten | 1.5588902321464466 | 
 | oezbekistan | 1.0017786861976063 | uitblijven | 0.7415464892478193 | resten | 0.9064642091914905 | normale | 0.7542116195739694 | verjaard | 1.930452001280508 | grootschalig | 0.3896423179577661 | geworven | 0.30255954304494675 | stroomaggregaten | 0.4346903879604562 | gunning | 0.9046081268620453 | betalingsysteem | 1.5588902321464466 | 
 | mensenrechtenactivisten | 1.0017786861976063 | doorgaat | 0.7415464892478193 | bentoniet | 0.9064642091914905 | middelbaar | 0.7542116195739694 | denkt | 1.930452001280508 | optredende | 0.3896423179577661 | resp | 0.30255954304494675 | defensieapparatuur | 0.4346903879604562 | fin | 0.9046081268620453 | definitie | 1.5588902321464466 | 
 | aanhef | 1.0017786861976063 | aardappelen | 0.7415464892478193 | traject | 0.9064642091914905 | wbo | 0.7542116195739694 | smeergeldaffaires | 1.930452001280508 | dierziekten | 0.3896423179577661 | slachtoffer | 0.30255954304494675 | goederen | 0.4346903879604562 | arbeidsongeschiktheidsuitkering | 0.9046081268620453 | geneesmiddelenvoorziening | 1.5588902321464466 | 
 | zigeuners | 1.0017786861976063 | teelt | 0.7415464892478193 | verdubbeling | 0.9064642091914905 | decentralisatie | 0.7542116195739694 | permanent | 1.930452001280508 | voorbeeld | 0.3896423179577661 | clusterbommen | 0.30255954304494675 | vergunningenstop | 0.4346903879604562 | zalm | 0.9046081268620453 | geneesmiddelen | 1.5588902321464466 | 
 | velen | 1.0017786861976063 | libertel | 0.7415464892478193 | landelijk | 0.9064642091914905 | basisonderwijs | 0.7542116195739694 | uitgaansgelegenheden | 1.930452001280508 | omvangrijke | 0.3896423179577661 | daarbij | 0.30255954304494675 | dynaf | 0.4346903879604562 | omscholingen | 0.9046081268620453 | afleveren | 1.5588902321464466 | 
 | serviers | 1.0017786861976063 | zendinstallaties | 0.7415464892478193 | baggerspecie | 0.9064642091914905 | oetc | 0.7542116195739694 | disco | 1.930452001280508 | voedselketen | 0.3896423179577661 | missie | 0.30255954304494675 | strategische | 0.4346903879604562 | vacatures | 0.9046081268620453 | uiteindelijke | 1.5588902321464466 | 
 | ontvlucht | 1.0017786861976063 | haarlemmermeer | 0.7415464892478193 | berging | 0.9064642091914905 | formatiebesluit | 0.7542116195739694 | gecontroleerd | 1.930452001280508 | pluimveehouders | 0.3896423179577661 | militairen | 0.30255954304494675 | energietechniek | 0.4346903879604562 | sollicitatietrainingen | 0.9046081268620453 | drogisten | 1.5588902321464466 | 
 | uitleveringsverzoek | 1.0017786861976063 | benw | 0.7415464892478193 | ijssel | 0.9064642091914905 | tegenstelling | 0.7542116195739694 | maat | 1.930452001280508 | standpunt | 0.3896423179577661 | opruiming | 0.30255954304494675 | oosterveen | 0.4346903879604562 | tilburg | 0.9046081268620453 | zelfzorgmedicijnen | 1.5588902321464466 | 
 | pais | 1.0017786861976063 | vonnis | 0.7415464892478193 | uitbaggeren | 0.9064642091914905 | terugdringen | 0.7542116195739694 | bezoekers | 1.930452001280508 | verontreinigd | 0.3896423179577661 | antipersoneelsmijnen | 0.30255954304494675 | inro | 0.4346903879604562 | tuinbouw | 0.9046081268620453 | recept | 1.5588902321464466 | 
 | el | 1.0017786861976063 | verwijderingsheffing | 0.7415464892478193 | hollandse | 0.9064642091914905 | bestrijden | 0.7542116195739694 | wegenverkeerswet | 1.930452001280508 | opgespoten | 0.3896423179577661 | detectie | 0.30255954304494675 | verwerkt | 0.4346903879604562 | poolse | 0.9046081268620453 | opdraaien | 1.5588902321464466 | 
 | annuleren | 1.0017786861976063 | milieuvriendelijke | 0.7415464892478193 | vaargeulen | 0.9064642091914905 | lerarentekort | 0.7542116195739694 | kellenaers | 1.930452001280508 | havenslib | 0.3896423179577661 | stress | 0.30255954304494675 | opbrengsten | 0.4346903879604562 | prof | 0.9046081268620453 | abortuspil | 1.5588902321464466 | 
 | procureur | 1.0017786861976063 | sloop | 0.7415464892478193 | verontreinigde | 0.9064642091914905 | oormerken | 0.7542116195739694 | arresten | 1.930452001280508 | begrazing | 0.3896423179577661 | verzorging | 0.30255954304494675 | standaardisatie | 0.4346903879604562 | scholing | 0.9046081268620453 | ru | 1.5588902321464466 | 
 | zichtbaarder | 1.0017786861976063 | mindere | 0.7415464892478193 | toeristische | 0.9064642091914905 | herintreders | 0.7542116195739694 | anja | 1.930452001280508 | veterinaire | 0.3896423179577661 | wervingscampagne | 0.30255954304494675 | bevat | 0.4346903879604562 | kbs | 0.9046081268620453 | mifygene | 1.5588902321464466 | 
 | politiegeweld | 1.0017786861976063 | grijs | 0.7415464892478193 | attracties | 0.9064642091914905 | lambrechts | 0.7542116195739694 | arrestatie | 1.930452001280508 | biodiesel | 0.3896423179577661 | ingestoken | 0.30255954304494675 | toeristenbelasting | 0.4346903879604562 | kra | 0.9046081268620453 | piercers | 1.5588902321464466 | 
 | rechtsvervolging | 1.0017786861976063 | kwaliteit | 0.7415464892478193 | pretparken | 0.9064642091914905 | enertel | 0.7542116195739694 | protestactie | 1.930452001280508 | koolzaad | 0.3896423179577661 | functionarissen | 0.30255954304494675 | heffingsgrondslagen | 0.4346903879604562 | arbeidsinpassing | 0.9046081268620453 | steriel | 1.5588902321464466 | 
 | overwegen | 1.0017786861976063 | leiding | 0.7415464892478193 | bereikbaarheid | 0.9064642091914905 | kapitaalkrachtige | 0.7542116195739694 | alarmteam | 1.930452001280508 | braakliggende | 0.3896423179577661 | reductie | 0.30255954304494675 | gemeentelijke | 0.4346903879604562 | genomen | 0.9046081268620453 | tatoeeerders | 1.5588902321464466 | 
 | chileense | 1.0017786861976063 | milieuwinst | 0.7415464892478193 | goederenspoorlijn | 0.9064642091914905 | opheffingsnorm | 0.7542116195739694 | waarnemers | 1.930452001280508 | geteeld | 0.3896423179577661 | bewaking | 0.30255954304494675 | splitsing | 0.4346903879604562 | bedrijfstakgewijze | 0.9046081268620453 | hygienisch | 1.5588902321464466 | 
 | stabiliseringsproces | 1.0017786861976063 | huishoudwater | 0.7415464892478193 | zestien | 0.9064642091914905 | zuideinde | 0.7542116195739694 | humanitaire | 1.930452001280508 | varkensbedrijven | 0.3896423179577661 | uittocht | 0.30255954304494675 | verlopen | 0.4346903879604562 | kaderregeling | 0.9046081268620453 | verandering | 1.5588902321464466 | 
 | buitenproportionele | 1.0017786861976063 | zwemwater | 0.7415464892478193 | reisservice | 0.9064642091914905 | voortbestaan | 0.7542116195739694 | helwijk | 1.930452001280508 | voodat | 0.3896423179577661 | zwaar | 0.30255954304494675 | steunkaart | 0.4346903879604562 | weten | 0.9046081268620453 | trekkingsrechten | 1.5588902321464466 | 
 | stad | 1.0017786861976063 | alternatieven | 0.7415464892478193 | handicap | 0.9064642091914905 | ijsselmuiden | 0.7542116195739694 | alsnog | 1.930452001280508 | onmisbaarheid | 0.3896423179577661 | grensbewaking | 0.30255954304494675 | moeizaam | 0.4346903879604562 | bijstand | 0.9046081268620453 | persoonsgebonden | 1.5588902321464466 | 
 | grozni | 1.0017786861976063 | chloordosering | 0.7415464892478193 | toegankelijkheid | 0.9064642091914905 | kamperveen | 0.7542116195739694 | verblijfsvergunning | 1.930452001280508 | beoordeling | 0.3896423179577661 | druk | 0.30255954304494675 | steunmaatregelen | 0.4346903879604562 | vries | 0.9046081268620453 | pgb | 1.5588902321464466 | 
 | hulpfonds | 1.0017786861976063 | milieuhygiëne | 0.7415464892478193 | vervoersvoorzieningen | 0.9064642091914905 | buitenkernen | 0.7542116195739694 | gronden | 1.930452001280508 | gewasbescherming | 0.3896423179577661 | massale | 0.30255954304494675 | hiermee | 0.4346903879604562 | bijzondere | 0.9046081268620453 | uitvoeringsregels | 1.5588902321464466 | 
 | azië | 1.0017786861976063 | boorlocaties | 0.7415464892478193 | singh | 0.9064642091914905 | optrekken | 0.7542116195739694 | genoeg | 1.930452001280508 | gespreksronde | 0.3896423179577661 | vreemdelingentoezicht | 0.30255954304494675 | multinationale | 0.4346903879604562 | afschrift | 0.9046081268620453 | otterloo | 1.5588902321464466 | 
 | ecip | 1.0017786861976063 | goedkeuring | 0.7415464892478193 | oedayraj | 0.9064642091914905 | cash | 0.7542116195739694 | korthals | 1.930452001280508 | pine | 0.3896423179577661 | landmachtpersoneel | 0.30255954304494675 | concerns | 0.4346903879604562 | nader | 0.9046081268620453 | toegelaten | 1.5588902321464466 | 
 | april | 1.0017786861976063 | buitengebied | 0.7415464892478193 | varma | 0.9064642091914905 | vermogen | 0.7542116195739694 | banden | 1.930452001280508 | company | 0.3896423179577661 | feddema | 0.30255954304494675 | bestelling | 0.4346903879604562 | circulaire | 0.9046081268620453 | drang | 1.5588902321464466 | 
 | vredesmacht | 1.0017786861976063 | onthouden | 0.7415464892478193 | giskes | 0.9064642091914905 | introductieweken | 0.7542116195739694 | smartshops | 1.930452001280508 | natuurwaarden | 0.3896423179577661 | tribune | 0.30255954304494675 | flessen | 0.4346903879604562 | ns | 0.9046081268620453 | lozing | 1.5588902321464466 | 
 | wesley | 1.0017786861976063 | gs | 0.7415464892478193 | vrachttoestel | 0.9064642091914905 | gepropageerd | 0.7542116195739694 | gedogen | 1.930452001280508 | natuurvriendenhuizen | 0.3896423179577661 | watergate | 0.30255954304494675 | gevestigde | 0.4346903879604562 | verzorgen | 0.9046081268620453 | zachte | 1.5588902321464466 | 
 | clark | 1.0017786861976063 | gedeputeerde | 0.7415464892478193 | overvliegend | 0.9064642091914905 | gegeven | 0.7542116195739694 | coffeeshop | 1.930452001280508 | exploitant | 0.3896423179577661 | hr | 0.30255954304494675 | persoon | 0.4346903879604562 | ambulances | 0.9046081268620453 | ribw | 1.5588902321464466 | 
 | michael | 1.0017786861976063 | eemsmond | 0.7415464892478193 | autoloze | 0.9064642091914905 | verkeersles | 0.7542116195739694 | zwijndrecht | 1.930452001280508 | patatvos | 0.3896423179577661 | rapporten | 0.30255954304494675 | groeiende | 0.4346903879604562 | twintig | 0.9046081268620453 | beschermd | 1.5588902321464466 | 
 | sir | 1.0017786861976063 | tc | 0.7415464892478193 | zondag | 0.9064642091914905 | ploeg | 0.7542116195739694 | uitzettingsbeleid | 1.930452001280508 | strandetablissementen | 0.3896423179577661 | soesterberg | 0.30255954304494675 | gedestilleerd | 0.4346903879604562 | raakte | 0.9046081268620453 | dakloze | 1.5588902321464466 | 
 | meningsverschillen | 1.0017786861976063 | aquilex | 0.7415464892478193 | milieubelasting | 0.9064642091914905 | fondsenbrief | 0.7542116195739694 | legaliseren | 1.930452001280508 | boulevards | 0.3896423179577661 | aanhoudend | 0.30255954304494675 | wettig | 0.4346903879604562 | evaluatie | 0.9046081268620453 | gezinnen | 1.5588902321464466 | 
 | aftreden | 1.0017786861976063 | megafusie | 0.7415464892478193 | toiletsystemen | 0.9064642091914905 | cruciale | 0.7542116195739694 | autodiefstal | 1.930452001280508 | vossen | 0.3896423179577661 | brandweerpersoneel | 0.30255954304494675 | accijnzen | 0.4346903879604562 | overleed | 0.9046081268620453 | schipperskinderen | 1.5588902321464466 | 
 | jackson | 1.0017786861976063 | gelderland | 0.7415464892478193 | toiletvoorzieningen | 0.9064642091914905 | kennisgeving | 0.7542116195739694 | moskou | 1.930452001280508 | bloemendaal | 0.3896423179577661 | asbestvervuiling | 0.30255954304494675 | geheven | 0.4346903879604562 | bekneld | 0.9046081268620453 | verhoging | 1.5588902321464466 | 
 | vervroegd | 1.0017786861976063 | waterleidingbedrijven | 0.7415464892478193 | lozingen | 0.9064642091914905 | fondsen | 0.7542116195739694 | rechercheurs | 1.930452001280508 | weidevogels | 0.3896423179577661 | peel | 0.30255954304494675 | maximaal | 0.4346903879604562 | rie | 0.9046081268620453 | ouderbijdrage | 1.5588902321464466 | 
 | materiaal | 1.0017786861976063 | loodfabriek | 0.7415464892478193 | bevorderen | 0.9064642091914905 | meldt | 0.7542116195739694 | punten | 1.930452001280508 | muizen | 0.3896423179577661 | geleide | 0.30255954304494675 | slijtersbedrijf | 0.4346903879604562 | werknemer | 0.9046081268620453 | internaten | 1.5588902321464466 | 
 | inheemse | 1.0017786861976063 | loodaccu | 0.7415464892478193 | pleziervaartuigen | 0.9064642091914905 | bleek | 0.7542116195739694 | strijdig | 1.930452001280508 | hazen | 0.3896423179577661 | ernst | 0.30255954304494675 | electriciteitscentrales | 0.4346903879604562 | betonelementen | 0.9046081268620453 | zwerfjongeren | 1.5588902321464466 | 
 | volken | 1.0017786861976063 | verwerking | 0.7415464892478193 | francisco | 0.9064642091914905 | leerlingbegeleiding | 0.7542116195739694 | rechtseenheidskamer | 1.930452001280508 | duinen | 0.3896423179577661 | wellicht | 0.30255954304494675 | borssele | 0.4346903879604562 | betonfabriek | 0.9046081268620453 | ambulancezorg | 1.5588902321464466 | 
 | dna | 1.0017786861976063 | milieuvoorschriften | 0.7415464892478193 | retour | 0.9064642091914905 | nieuws | 0.7542116195739694 | rek | 1.930452001280508 | voedsel | 0.3896423179577661 | gaande | 0.30255954304494675 | beslissingen | 0.4346903879604562 | bedrijfsongeval | 0.9046081268620453 | contact | 1.5588902321464466 | 
 | kofi | 1.0017786861976063 | blanc | 0.7415464892478193 | duur | 0.9064642091914905 | avbb | 0.7542116195739694 | argos | 1.930452001280508 | gesignaleerd | 0.3896423179577661 | volkel | 0.30255954304494675 | kerncentrale | 0.4346903879604562 | borne | 0.9046081268620453 | tweedelijnsarbozorg | 1.5588902321464466 | 
 | sanctiecommissie | 1.0017786861976063 | arbeidsomstandigheden | 0.7415464892478193 | afstand | 0.9064642091914905 | instantie | 0.7542116195739694 | meerdere | 1.930452001280508 | konijnen | 0.3896423179577661 | bond | 0.30255954304494675 | ontmoeten | 0.4346903879604562 | scholingsinstituut | 0.9046081268620453 | cagin | 1.5588902321464466 | 
 | annan | 1.0017786861976063 | verbruikte | 0.7415464892478193 | klm | 0.9064642091914905 | schooljournaal | 0.7542116195739694 | gevonden | 1.930452001280508 | vormen | 0.3896423179577661 | abvakabo | 0.30255954304494675 | delegaties | 0.4346903879604562 | erkende | 0.9046081268620453 | bemiddelt | 1.5588902321464466 | 
 | veiligheidspolitie | 1.0017786861976063 | overtreden | 0.7415464892478193 | km | 0.9064642091914905 | elkaars | 0.7542116195739694 | dierbevrijdingsfront | 1.930452001280508 | trekt | 0.3896423179577661 | werkzaam | 0.30255954304494675 | organization | 0.4346903879604562 | eega | 0.9046081268620453 | arbeidsgerelateerde | 1.5588902321464466 | 
 | xian | 1.0017786861976063 | metal | 0.7415464892478193 | korter | 0.9064642091914905 | salaris | 0.7542116195739694 | beursfraudezaken | 1.930452001280508 | zandvoort | 0.3896423179577661 | belastingdienst | 0.30255954304494675 | bestond | 0.4346903879604562 | inleveren | 0.9046081268620453 | omzeilt | 1.5588902321464466 | 
 | staatsveiligheidsdiensten | 1.0017786861976063 | spoor | 0.7415464892478193 | gladheidsbestrijding | 0.9064642091914905 | betere | 0.7542116195739694 | fred | 1.930452001280508 | hormoonwetgeving | 0.3896423179577661 | konvooi | 0.30255954304494675 | diners | 0.4346903879604562 | uitkeringsgerechtigde | 0.9046081268620453 | aandoeningen | 1.5588902321464466 | 
 | plaatselijke | 1.0017786861976063 | diesellocs | 0.7415464892478193 | asfaltbeton | 0.9064642091914905 | secundaire | 0.7542116195739694 | versterking | 1.930452001280508 | hormoneneenheid | 0.3896423179577661 | vluchtelingen | 0.30255954304494675 | staatshoofden | 0.4346903879604562 | specificeren | 0.9046081268620453 | tabaksindustrie | 1.5588902321464466 | 
 | henan | 1.0017786861976063 | verslag | 0.7415464892478193 | zoab | 0.9064642091914905 | dreigt | 0.7542116195739694 | cohen | 1.930452001280508 | vluchtroute | 0.3896423179577661 | orahovac | 0.30255954304494675 | wto | 0.4346903879604562 | inkomstenformulier | 0.9046081268620453 | toevoegt | 1.5588902321464466 | 
 | betrokkenheid | 1.0017786861976063 | sp | 0.7415464892478193 | winterse | 0.9064642091914905 | stoten | 0.7542116195739694 | legalisering | 1.930452001280508 | hormoonmaffia | 0.3896423179577661 | actief | 0.30255954304494675 | host | 0.4346903879604562 | wychen | 0.9046081268620453 | controlemaatregelen | 1.5588902321464466 | 
 | studentendemonstraties | 1.0017786861976063 | huurverhoging | 0.7415464892478193 | methode | 0.9064642091914905 | vnu | 0.7542116195739694 | verwijst | 1.930452001280508 | inspectiedienst | 0.3896423179577661 | gmelich | 0.30255954304494675 | recepties | 0.4346903879604562 | dag | 0.9046081268620453 | veevoer | 1.5588902321464466 | 
 | mission | 1.0017786861976063 | bankiers | 0.7415464892478193 | casestudie | 0.9064642091914905 | verschraling | 0.7542116195739694 | jus | 1.930452001280508 | aid | 0.3896423179577661 | lobbyist | 0.30255954304494675 | statistieken | 0.4346903879604562 | bureaus | 0.9046081268620453 | levensmiddelen | 1.5588902321464466 | 
 | vacuum | 1.0017786861976063 | syndicaten | 0.7415464892478193 | veiligheids | 0.9064642091914905 | procent | 0.7542116195739694 | politiecel | 1.930452001280508 | fokprogramma | 0.3896423179577661 | samenhangt | 0.30255954304494675 | betrouwbare | 0.4346903879604562 | banenprojecten | 0.9046081268620453 | wegkopen | 1.5588902321464466 | 
 | training | 1.0017786861976063 | staal | 0.7415464892478193 | verder | 0.9064642091914905 | sturen | 0.7542116195739694 | opgesloten | 1.930452001280508 | korenwolven | 0.3896423179577661 | meijling | 0.30255954304494675 | national | 0.4346903879604562 | stopzetting | 0.9046081268620453 | gespecialiseerd | 1.5588902321464466 | 
 | establishment | 1.0017786861976063 | waarde | 0.7415464892478193 | verkeersleiding | 0.9064642091914905 | kilometer | 0.7542116195739694 | toegewezen | 1.930452001280508 | zetten | 0.3896423179577661 | brengt | 0.30255954304494675 | wereldwijd | 0.4346903879604562 | patstelling | 0.9046081268620453 | transfersom | 1.5588902321464466 | 
 | onmiddellijk | 1.0017786861976063 | principe | 0.7415464892478193 | luchthavens | 0.9064642091914905 | verderop | 0.7542116195739694 | mislopen | 1.930452001280508 | gevangenschap | 0.3896423179577661 | gevoelige | 0.30255954304494675 | nsa | 0.4346903879604562 | vakantieduur | 0.9046081268620453 | opgelopen | 1.5588902321464466 | 
 | filling | 1.0017786861976063 | woningcorporatie | 0.7415464892478193 | overheidsbijdragen | 0.9064642091914905 | buurt | 0.7542116195739694 | plek | 1.930452001280508 | vangen | 0.3896423179577661 | apachehelikopters | 0.30255954304494675 | monde | 0.4346903879604562 | verwarring | 0.9046081268620453 | reguliere | 1.5588902321464466 | 
 | plaatse | 1.0017786861976063 | nv | 0.7415464892478193 | vliegtickets | 0.9064642091914905 | eren | 0.7542116195739694 | voordoen | 1.930452001280508 | use | 0.3896423179577661 | pelotons | 0.30255954304494675 | security | 0.4346903879604562 | gebruikelijke | 0.9046081268620453 | opname | 1.5588902321464466 | 
 | local | 1.0017786861976063 | vastgoed | 0.7415464892478193 | prijzen | 0.9064642091914905 | ervoor | 0.7542116195739694 | scheiden | 1.930452001280508 | restriction | 0.3896423179577661 | eenheid | 0.30255954304494675 | geeft | 0.4346903879604562 | werkgeversfraude | 0.9046081268620453 | vrijlaten | 1.5588902321464466 | 
 | leadership | 1.0017786861976063 | tegenwoordig | 0.7415464892478193 | vergelijking | 0.9064642091914905 | techniek | 0.7542116195739694 | uiteenlopende | 1.930452001280508 | gurt | 0.3896423179577661 | mobiele | 0.30255954304494675 | le | 0.4346903879604562 | flexwerkers | 0.9046081268620453 | medicijnenprijzen | 1.5588902321464466 | 
 | power | 1.0017786861976063 | basco | 0.7415464892478193 | vliegmaatschappijen | 0.9064642091914905 | afneemt | 0.7542116195739694 | asiel | 1.930452001280508 | kieming | 0.3896423179577661 | opleiding | 0.30255954304494675 | nt | 0.4346903879604562 | werkgevers | 0.9046081268620453 | adviseur | 1.5588902321464466 | 
 | interin | 1.0017786861976063 | ingestemd | 0.7415464892478193 | vermelden | 0.9064642091914905 | voortzetting | 0.7542116195739694 | reisroutes | 1.930452001280508 | samenleving | 0.3896423179577661 | inzetbare | 0.30255954304494675 | windows | 0.4346903879604562 | arbeidsbureaus | 0.9046081268620453 | borst | 1.5588902321464466 | 
 | municipal | 1.0017786861976063 | smeersubsidies | 0.7415464892478193 | reisorganisaties | 0.9064642091914905 | wetenschapswinkel | 0.7542116195739694 | vestiging | 1.930452001280508 | term | 0.3896423179577661 | vorming | 0.30255954304494675 | agency | 0.4346903879604562 | slecht | 0.9046081268620453 | koopmans | 1.5588902321464466 | 
 | benoemen | 1.0017786861976063 | stem | 0.7415464892478193 | zandwinningsbedrijven | 0.9064642091914905 | monumentwaardige | 0.7542116195739694 | deurwaarders | 1.930452001280508 | wetenschappelijke | 0.3896423179577661 | bestemmen | 0.30255954304494675 | encryptiesleutel | 0.4346903879604562 | boer | 0.9046081268620453 | verzekerd | 1.5588902321464466 | 
 | authorities | 1.0017786861976063 | breau | 0.7415464892478193 | nma | 0.9064642091914905 | kerkgebouwen | 0.7542116195739694 | gerechtsdeurwaarders | 1.930452001280508 | technologies | 0.3896423179577661 | afstoting | 0.30255954304494675 | elektriciteitsvoorziening | 0.4346903879604562 | ingeschreven | 0.9046081268620453 | waarvoor | 1.5588902321464466 | 
 | unmik | 1.0017786861976063 | focus | 0.7415464892478193 | mededingingsautoriteit | 0.9064642091914905 | economie | 0.7542116195739694 | redelijke | 1.930452001280508 | benaming | 0.3896423179577661 | weert | 0.30255954304494675 | toezicht | 0.4346903879604562 | arbeidsproces | 0.9046081268620453 | huishoudelijke | 1.5588902321464466 | 
 | processen | 1.0017786861976063 | delftse | 0.7415464892478193 | zandwinprojecten | 0.9064642091914905 | krappe | 0.7542116195739694 | rechtszaak | 1.930452001280508 | vaak | 0.3896423179577661 | marinevliegkamp | 0.30255954304494675 | marktwerking | 0.4346903879604562 | avondopleiding | 0.9046081268620453 | wachtlijst | 1.5588902321464466 | 
 | geweldsuitbarstingen | 1.0017786861976063 | transformatoren | 0.7415464892478193 | vallen | 0.9064642091914905 | rijksbijdrage | 0.7542116195739694 | verklaard | 1.930452001280508 | genetic | 0.3896423179577661 | extended | 0.30255954304494675 | dte | 0.4346903879604562 | gebarentaal | 0.9046081268620453 | oversterfte | 1.5588902321464466 | 
 | ogoniland | 1.0017786861976063 | crone | 0.7415464892478193 | geraamd | 0.9064642091914905 | ravestein | 0.7542116195739694 | verstrijken | 1.930452001280508 | zaad | 0.3896423179577661 | task | 0.30255954304494675 | hakvoort | 0.4346903879604562 | doventolk | 0.9046081268620453 | optreedt | 1.5588902321464466 | 
 | daarop | 1.0017786861976063 | verwijderingsbijdrage | 0.7415464892478193 | sluizen | 0.9064642091914905 | systematiek | 0.7542116195739694 | bloed | 1.930452001280508 | vliegveld | 0.3896423179577661 | valkenburg | 0.30255954304494675 | managementteam | 0.4346903879604562 | dove | 0.9046081268620453 | hittegolf | 1.5588902321464466 | 
 | srebrenica | 1.0017786861976063 | kpmg | 0.7415464892478193 | julianakanaal | 0.9064642091914905 | kerkbesturen | 0.7542116195739694 | alcohol | 1.930452001280508 | terlet | 0.3896423179577661 | hoofdkwartier | 0.30255954304494675 | graydon | 0.4346903879604562 | ingepast | 0.9046081268620453 | ca | 1.5588902321464466 | 
 | gehouden | 1.0017786861976063 | bedrijfspanden | 0.7415464892478193 | stijgers | 0.9064642091914905 | ontheffing | 0.7542116195739694 | promillagegrens | 1.930452001280508 | terletse | 0.3896423179577661 | kamercommissie | 0.30255954304494675 | detailhandels | 0.4346903879604562 | mondjesmaat | 0.9046081268620453 | overhouden | 1.5588902321464466 | 
 | onlangs | 1.0017786861976063 | centrales | 0.7415464892478193 | drijvende | 0.9064642091914905 | doorberekenen | 0.7542116195739694 | camp | 1.930452001280508 | heide | 0.3896423179577661 | keulen | 0.30255954304494675 | verduidelijking | 0.4346903879604562 | halveren | 0.9046081268620453 | kleedgeld | 1.5588902321464466 | 
 | leider | 1.0017786861976063 | zonnepanelen | 0.7415464892478193 | ongevraagd | 0.9064642091914905 | compenseren | 0.7542116195739694 | oven | 1.930452001280508 | geef | 0.3896423179577661 | defence | 0.30255954304494675 | vestigingsbeleid | 0.4346903879604562 | streefcijfer | 0.9046081268620453 | maandelijkse | 1.5588902321464466 | 
 | voortgang | 1.0017786861976063 | prijs | 0.7415464892478193 | telecom | 0.9064642091914905 | kostenstijging | 0.7542116195739694 | hoeveelheden | 1.930452001280508 | erfpacht | 0.3896423179577661 | oprichtingsceremonie | 0.30255954304494675 | grootschalige | 0.4346903879604562 | helpt | 0.9046081268620453 | gedaald | 1.5588902321464466 | 
 | rusland | 1.0017786861976063 | nieuwbouw | 0.7415464892478193 | actueel | 0.9064642091914905 | btw | 0.7542116195739694 | dierenbevrijdingsfront | 1.930452001280508 | zweefvliegcentrum | 0.3896423179577661 | force | 0.30255954304494675 | geconcentreerde | 0.4346903879604562 | reïntegreren | 0.9046081268620453 | aow | 1.5588902321464466 | 


## 6.
Evaluate your classifiers using Precision, Recall and F1. (Give a table in which you show these values for using the top 10, top 100 terms and all terms, for all of your 10 classes)
          Thus do feature selection per class, and use for each class the top n best features for that class. 
          <br/>
      Also show the microaverage(s) for all 10 classes together.
      <br/>
      If you like you can also present this in a figure like MRS.13.8. 
      Then compute the F1 measure for the same number of terms as in that figure.

In [ ]:


def precision():
    raise NotImplemented
    
def recall():
    raise NotImplemented
    
def F1_measure():
    P = precision()
    R = recall()
    return 2*P*R/(P+R)

In [ ]:
def getstats()
    ApplyMultinomialNB(classes, priors, conditionals, d, Vocab=None)

## 7 
You have done the complete implementation by yourself. Congratulations! You can also use `scikit-learn` routines for all of this work. Do that. So follow [this text classification tutorial](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)  and implement the same steps but now with your kamervragen dataset. Also use [mutual information feature selection](http://scikit-learn.org/stable/modules/feature_selection.html) to select the K-best features, and compare the results as before.


## 8

Reflect and report briefly about your choices in this process and about the obtained results. Also reflect on the differences between the scikit learn approach and the "own implementation approach".

<h3>Training/Testing</h3>
<p>It is important that you do not test your classifier using documents that have also been used in training.
    So split up your collection in a training set and a test set. A 80%-20% split is reasonable.

<br/>
    If you have too little data you can use 5 or <a href="http://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation">10-fold cross validation</a>.</p>



<h2>Form of handing in your final product</h2>

* An IPython notebook with for each question, a MarkDown cell containing the question, a code cell which solves the question, an output cell with the output, followed by a MarkDown cell with explanation/reflection  